In [2]:
import pandas as pd
import numpy as np
import pickle

freq_termdoc = None
with open("frequency_termdoc.pkl", "rb") as f:
	freq_termdoc = pickle.load(f)

bin_termdoc = None
with open("binary_termdoc.pkl", "rb") as f:
	bin_termdoc = pickle.load(f)

In [3]:
freq_svd = np.linalg.svd(freq_termdoc)
bin_svd = np.linalg.svd(bin_termdoc)

In [6]:
with open("freq_termdoc_svd.pkl", "wb") as f:
	pickle.dump(freq_svd, f)

with open("bin_termdoc_svd.pkl", "wb") as f:
	pickle.dump(bin_svd, f)

In [4]:
M, N = freq_termdoc.shape
M, N

(10115, 346)

### Frequency Based TermDoc

Values are pretty consistently similar in magnitude until the last two guys, so we should maybe have $K = N - 2$, but that would likely not give us a good shrink in rank (i.e. no compression of "semantic meaning" to deal with polysemy and synonymy), so I pick an arbitrary shrink, $K = N // 2$, and take the best K rank approximation to A by truncating the SVD to k terms

In [18]:
K = N // 2

In [19]:
Uk = np.copy(freq_svd.U)
Uk[:, K:] = 0

Sk = np.copy(freq_svd.S)
Sk[K:] = 0

Vhk = np.copy(freq_svd.Vh)
Vhk[K:, :] = 0

In [20]:
with open("freq_lowrank.pkl", "wb") as f:
	pickle.dump({"Uk": Uk, "Sk": Sk, "Vhk": Vhk}, f)

### Binary Termdoc
Same deal

In [21]:
K = N // 2

In [22]:
Uk = np.copy(bin_svd.U)
Uk[:, K:] = 0

Sk = np.copy(bin_svd.S)
Sk[K:] = 0

Vhk = np.copy(bin_svd.Vh)
Vhk[K:, :] = 0

In [23]:
with open("bin_lowrank.pkl", "wb") as f:
	pickle.dump({"Uk": Uk, "Sk": Sk, "Vhk": Vhk}, f)